In [1]:
import pandas as pd
from pathlib import Path

In [2]:
!pip install xgboost

import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
import xgboost as xgb
from sklearn.metrics import accuracy_score


In [3]:
train= pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test= pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sub= pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')


In [4]:
train['sex'] = train['sex'].fillna('na')
train['age_approx'] = train['age_approx'].fillna(0)
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].fillna('na')

test['sex'] = test['sex'].fillna('na')
test['age_approx'] = test['age_approx'].fillna(0)
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].fillna('na')
train['sex'] = train['sex'].astype("category").cat.codes +1
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].astype("category").cat.codes +1
train.head()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,2,45.0,1,unknown,benign,0
1,ISIC_0015719,IP_3075186,1,45.0,7,unknown,benign,0
2,ISIC_0068279,IP_6890425,1,45.0,1,unknown,benign,0
3,ISIC_0074268,IP_8723313,1,55.0,7,unknown,benign,0
4,ISIC_0074311,IP_2950485,1,40.0,2,unknown,benign,0


In [5]:
test['sex'] = test['sex'].astype("category").cat.codes +1
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].astype("category").cat.codes +1
test.head()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052212,IP_2842074,1,50.0,2
1,ISIC_0076545,IP_9802602,2,55.0,7
2,ISIC_0085172,IP_1705144,1,50.0,2
3,ISIC_0086709,IP_4109313,2,30.0,6
4,ISIC_0109568,IP_0825081,2,80.0,2


In [6]:
x_train = train[['sex', 'age_approx','anatom_site_general_challenge']]
y_train = train['target']


x_test = test[['sex', 'age_approx','anatom_site_general_challenge']]
# y_train = test['target']


train_DMatrix = xgb.DMatrix(x_train, label= y_train)
test_DMatrix = xgb.DMatrix(x_test)


In [7]:
clf = xgb.XGBClassifier(n_estimators=2600, 
                        max_depth=20, 
                        objective='multi:softprob',
                        seed=0,  
                        nthread=-1, 
                        learning_rate=0.15, 
                        num_class = 2, 
                        scale_pos_weight = (32542/584))

clf.fit(x_train, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [03:19:34] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=20, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=2600, n_jobs=None, nthread=-1,
              num_class=2, ...)

In [8]:
clf.predict_proba(x_test)[:,1]
# clf.predict(x_test)
sub.target = clf.predict_proba(x_test)[:,1]
sub_tabular = sub.copy()


In [9]:
#sub_tabular.to_csv('submission.csv', index = False)


In [10]:
##Ensemble for 0.9565
# sub_new = pd.read_csv('../input/siimisic/sub-new.csv')
# sub_mean = pd.read_csv('/kaggle/input/siimisic/submission_mean.csv')
# sub.target = sub_mean.target *0.2 + sub_new.target *0.6 + sub_tabular.target *0.2
# sub.head()


In [11]:
sub_tabular


,image_name,target
0,ISIC_0052212,0.007407
1,ISIC_0076545,0.013527
2,ISIC_0085172,0.007407
3,ISIC_0086709,0.010779
4,ISIC_0109568,0.000360
...,...,...
4137,ISIC_9987485,0.011101
4138,ISIC_9989936,0.007067
4139,ISIC_9994122,0.055043
4140,ISIC_9994390,0.034579


In [12]:
ens_sub = pd.read_csv('../input/actual-output/submission.csv')
ens_sub.to_csv('ens_sub.csv', index = False)


FileNotFoundError: [Errno 2] No such file or directory: '../input/actual-output/submission.csv'

In [13]:
sub['target'] = (ens_sub.target - sub_tabular.target*0.2)/0.8
sub.to_csv('efn_sub.csv', index = False)


NameError: name 'ens_sub' is not defined

In [14]:
'''from scipy import stats
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import f1_score

clf_xgb = XGBClassifier(objective = 'binary:logistic')
param_dist = {'n_estimators': stats.randint(150, 500),
              'learning_rate': stats.uniform(0.01, 0.07),
              'subsample': stats.uniform(0.3, 0.7),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.45),
              'min_child_weight': [1, 2, 3]
             }
clf = RandomizedSearchCV(clf_xgb, param_distributions = param_dist, n_iter = 25, scoring = 'f1', error_score = 0, verbose = 3, n_jobs = -1)

numFolds = 5
folds = KFold(n_splits = numFolds, shuffle = True)

estimators = []
results = np.zeros(len(x_train))
score = 0.0
for train_index, test_index in folds.split(x_train):
    X_train, X_test = x_train[train_index,:], x_train.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index].values.ravel(), y.iloc[test_index].values.ravel()
    clf.fit(X_train, y_train)

    estimators.append(clf.best_estimator_)
    results[test_index] = clf.predict(X_test)
    score += f1_score(y_test, results[test_index])
score /= numFolds'''


"from scipy import stats\nfrom xgboost import XGBClassifier\nfrom sklearn.model_selection import RandomizedSearchCV, KFold\nfrom sklearn.metrics import f1_score\n\nclf_xgb = XGBClassifier(objective = 'binary:logistic')\nparam_dist = {'n_estimators': stats.randint(150, 500),\n              'learning_rate': stats.uniform(0.01, 0.07),\n              'subsample': stats.uniform(0.3, 0.7),\n              'max_depth': [3, 4, 5, 6, 7, 8, 9],\n              'colsample_bytree': stats.uniform(0.5, 0.45),\n              'min_child_weight': [1, 2, 3]\n             }\nclf = RandomizedSearchCV(clf_xgb, param_distributions = param_dist, n_iter = 25, scoring = 'f1', error_score = 0, verbose = 3, n_jobs = -1)\n\nnumFolds = 5\nfolds = KFold(n_splits = numFolds, shuffle = True)\n\nestimators = []\nresults = np.zeros(len(x_train))\nscore = 0.0\nfor train_index, test_index in folds.split(x_train):\n    X_train, X_test = x_train[train_index,:], x_train.iloc[test_index,:]\n    y_train, y_test = y.iloc[train_i

In [15]:
from scipy import stats
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import f1_score

clf_xgb = XGBClassifier(objective = 'multi:softprob')
param_dist = {'n_estimators': stats.randint(150, 500),
              'learning_rate': stats.uniform(0.01, 0.07),
              'subsample': stats.uniform(0.3, 0.7),
               'num_class':  (2,3),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.45),
              'min_child_weight': [1, 2, 3]
             }
clf = RandomizedSearchCV(clf_xgb, param_distributions = param_dist, n_iter = 25, scoring = 'f1', error_score = 0, verbose = 3, n_jobs = -1)


In [16]:
clf.fit(x_train, y_train)


Fitting 5 folds for each of 25 candidates, totalling 125 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
           ^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
           ^^^^^^^^^^^^
  File "/usr/lo

RandomizedSearchCV(error_score=0,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           lear...
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f5e73779fd0>,
                                        'max_depth': [3, 4, 5, 6, 7, 8, 9],
                                        'min_child_weight': [1, 2, 3],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f5e726dc750>,
                                        'num_class': (2, 3),
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f5e726dcf10>},
                   scoring='f1', verbose=3)

In [17]:
clf.best_params_


{'colsample_bytree': 0.833668053869139,
 'learning_rate': 0.04414805844484786,
 'max_depth': 5,
 'min_child_weight': 2,
 'n_estimators': 484,
 'num_class': 3,
 'subsample': 0.7979728437873695}

In [18]:
clf1 = xgb.XGBClassifier(n_estimators=432, 
                        max_depth=7, 
                        objective='multi:softprob',
                        seed=0,  
                        nthread=-1, 
                        learning_rate=0.015573109334896062,
                        num_class = 2, 
                        scale_pos_weight = (32542/584))

clf1.fit(x_train, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [03:20:08] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.015573109334896062,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=432, n_jobs=None, nthread=-1,
              num_class=2, ...)

In [19]:
clf.predict_proba(x_test)[:,1]
# clf.predict(x_test)
sub.target = clf.predict_proba(x_test)[:,1]
sub_tabular = sub.copy()


In [20]:
sub_tabular.to_csv('submission.csv', index=False)
